# Optuna pour PhysAE
Ce notebook fournit trois scénarios prêts à l'emploi pour lancer des optimisations Optuna : stage A seul, stage B (B1+B2) et la chaîne A→B avec possibilité d'affiner les résultats.
Utilisez les cellules suivantes pour configurer vos essais.

In [ ]:
import json
import pandas as pd
from physae.parameter_catalog import describe_parameters, list_stage_parameters
from physae.simple_workflows import OptunaRunConfig, run_optuna

## Paramètres par stage
Référence rapide des hyperparamètres optimisables.

In [ ]:
stage_tables = {name: pd.DataFrame(describe_parameters(list_stage_parameters(name))) for name in ("A", "B1", "B2")}
stage_tables

## Overrides globaux pour les données
Ajustez les bornes ou la taille des jeux si nécessaire.

In [ ]:
data_overrides = {
    # "batch_size": 32,
    # "n_train": 60000,
}


## Overrides spécifiques à un stage
Les clés suivent la notation `stage:chemin`. Exemple : `"B2:epochs": 20`.

In [ ]:
stage_overrides = {
    # "A": {"epochs": 25},
    # "B2": {"delta_scale": 0.09},
}


## Scénario 1 : Optuna sur le stage A
Lancez l'optimisation sur 20 essais (modifiable via `n_trials`).

In [ ]:
config_A = OptunaRunConfig(
    stages=["A"],
    n_trials=20,
    metric="val_loss",
    direction="minimize",
    data_overrides=data_overrides,
    stage_overrides=stage_overrides,
    sampler="tpe",
    pruner="median",
    output_dir="optuna_stage_A",
)
results_A, cumulative_A = run_optuna(config_A)
{stage: study.study.best_value for stage, study in {k: v.study for k, v in results_A.items()}.items()}, cumulative_A

## Scénario 2 : Optuna sur le stage B (B1+B2)
Le stage logique `B` optimise successivement B1 puis B2 en réutilisant les meilleurs paramètres.

In [ ]:
config_B = OptunaRunConfig(
    stages=["B"],
    n_trials=15,
    metric="val_loss",
    direction="minimize",
    data_overrides=data_overrides,
    stage_overrides=stage_overrides,
    sampler="tpe",
    pruner=None,
    output_dir="optuna_stage_B",
)
results_B, cumulative_B = run_optuna(config_B)
{stage: study.study.best_value for stage, study in {k: v.study for k, v in results_B.items()}.items()}, cumulative_B

## Scénario 3 : Chaîne A + B avec fine-tuning B2
Cette configuration optimise d'abord A puis B et renvoie les overrides cumulés pour pouvoir lancer un fine-tuning manuel sur B2.

In [ ]:
config_AB = OptunaRunConfig(
    stages=["A", "B"],
    n_trials=10,
    metric="val_loss",
    direction="minimize",
    data_overrides=data_overrides,
    stage_overrides=stage_overrides,
    sampler="random",
    pruner="successivehalving",
    output_dir="optuna_stage_AB",
)
results_AB, cumulative_AB = run_optuna(config_AB)
{stage: study.study.best_value for stage, study in {k: v.study for k, v in results_AB.items()}.items()}, cumulative_AB